In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
from PIL import Image

In [2]:
# Загрузка данных
train_data = pd.read_csv("train_data.csv")
test_data = pd.read_csv("test_data.csv")

In [3]:
# Предобработка данных
# Функция для чтения и предобработки изображений
def preprocess_image(image_path, target_size=(300, 300)):
    # Загрузка изображения
    img = load_img(image_path, target_size=target_size)
    # Преобразование изображения в массив numpy
    img_array = img_to_array(img)
    # Нормализация значений пикселей к диапазону [0, 1]
    img_array = img_array / 255.0
    return img_array

# Используем функцию preprocess_image для предобработки всех изображений из датасета
def preprocess_images(dataframe):
    processed_images = []
    for image_path in dataframe['image_path']:
        processed_image = preprocess_image(image_path)
        processed_images.append(processed_image)
    return np.array(processed_images)

# Применяем функцию предобработки к тренировочным и тестовым данным
train_images = preprocess_images(train_data)
test_images = preprocess_images(test_data)

# Получаем количество изображений в тренировочном и тестовом наборах
num_train_images = train_images.shape[0]
num_test_images = test_images.shape[0]

print("Количество изображений в тренировочном наборе:", num_train_images)
print("Количество изображений в тестовом наборе:", num_test_images)

Количество изображений в тренировочном наборе: 6194
Количество изображений в тестовом наборе: 1554


In [7]:
# Создаем экземпляры ImageDataGenerator для аугментации тренировочных и тестовых изображений
train_datagen = ImageDataGenerator(
    rescale=1./255,            # Перенормировка значений пикселей
    rotation_range=20,         # Угол поворота (в градусах)
    width_shift_range=0.2,     # Сдвиг по ширине
    height_shift_range=0.2,    # Сдвиг по высоте
    shear_range=0.2,           # Направление сдвига
    zoom_range=0.2,            # Масштабирование
    horizontal_flip=True,      # Отражение по горизонтали
    fill_mode='nearest'        # Заполнение пикселей за пределами границ изображения
)

test_datagen = ImageDataGenerator(rescale=1./255)  # Только перенормировка для тестовых данных

# Создаем генераторы данных для тренировочного и тестового наборов
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    x_col='image_path',            # Имя столбца с путями к изображениям
    y_col='label',                 # Имя столбца с метками классов
    target_size=(150, 150),        # Размер изображений (высота, ширина)
    batch_size=32,                 # Размер пакета данных
    class_mode='categorical'       # Режим классификации (категориальная классификация)
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    x_col='image_path',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 6194 validated image filenames belonging to 10 classes.
Found 1554 validated image filenames belonging to 10 classes.


In [17]:
# Получаем словарь классов и их индексов
class_indices = train_generator.class_indices

# Получаем количество уникальных классов
num_classes = len(class_indices)

# Выводим количество уникальных классов
print("Количество уникальных классов:", num_classes)

Количество уникальных классов: 10


In [9]:
# Создаем последовательную модель
model = Sequential()

# Добавляем сверточные слои с пулингом
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Добавляем слой для сглаживания данных перед подачей на полносвязные слои
model.add(Flatten())

# Добавляем полносвязные слои
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))  # Добавляем слой Dropout для уменьшения переобучения
model.add(Dense(num_classes, activation='softmax'))  # Выходной слой с softmax для классификации


C:\Users\14488\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [10]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [11]:
# Обучение модели
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)

Epoch 1/10


C:\Users\14488\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


194/194 ━━━━━━━━━━━━━━━━━━━━ 100s 504ms/step - accuracy: 0.1966 - loss: 2.2036 - val_accuracy: 0.2709 - val_loss: 2.1143
Epoch 2/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 151us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/10


C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1008.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


194/194 ━━━━━━━━━━━━━━━━━━━━ 96s 489ms/step - accuracy: 0.2767 - loss: 2.0839 - val_accuracy: 0.3211 - val_loss: 2.0228
Epoch 4/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 70us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 95s 483ms/step - accuracy: 0.3107 - loss: 2.0223 - val_accuracy: 0.3301 - val_loss: 1.9737
Epoch 6/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 18us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 93s 474ms/step - accuracy: 0.3233 - loss: 1.9810 - val_accuracy: 0.3378 - val_loss: 1.9240
Epoch 8/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 50us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 95s 483ms/step - accuracy: 0.3174 - loss: 1.9796 - val_accuracy: 0.3372 - val_loss: 1.9235
Epoch 10/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 63us/ste

In [13]:
# Оценка производительности модели на тестовых данных:

test_loss, test_accuracy = model.evaluate(test_generator, verbose=2)
print(f'Потери на тестовых данных: {test_loss}')
print(f'Точность на тестовых данных: {test_accuracy}')

49/49 - 7s - 133ms/step - accuracy: 0.3372 - loss: 1.9235
Потери на тестовых данных: 1.923471212387085
Точность на тестовых данных: 0.3371943235397339
